In [45]:
import random, re
from collections import defaultdict
import itertools
import pycosat
import matplotlib.pyplot as plt
import pandas as pd

In [46]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, BasicAer, IBMQ, execute
from qiskit.tools.visualization import plot_histogram

In [47]:
def tcnfgen(n,k,horn=1):
    cnf = []
    def unique(l,k):
        t = random.randint(1,n)
        while(t in l):
            t = random.randint(1,n)
        return t
    r = (lambda : random.randint(0,1))
    def r_to_sign(x):
        if r() == 1:
            return x
        else:
            return -x
    for i in range(k):
        x = unique([],n)
        y = unique([x],n)
        z = unique([x, y],n)
        if horn:
            cnf.append([x, -y,-z])
        else:
            cnf.append([r_to_sign(x), r_to_sign(y),r_to_sign(z)])
    return cnf

In [48]:
def create_CNF(n, k):
    new_line = []
    line = tcnfgen(n, k)
    for clause in line:
        new_line.append(sorted(clause, key = abs))
    return new_line

In [49]:
def prepare_clause(circuit, quantum_reg, clause):
    for element in clause:
        if element<0:
            circuit.x(quantum_reg[abs(element)])
    return circuit

In [50]:
def or_operator(circuit,quantum_reg, clause, counter):
    order_clause = [abs(x) for x in clause]
    for element in order_clause:
        circuit.x(quantum_reg[element])
    circuit.mcx(order_clause, quantum_reg[counter])
    circuit.x(quantum_reg[counter])
    for element in order_clause:
        circuit.x(quantum_reg[element])
    return circuit

In [51]:
def oracle(circuit, n, k,quantum_reg, CNF):
    counter = n+1 
    for clause in CNF:
        prepare_clause(circuit,quantum_reg, clause)
        or_operator(circuit,quantum_reg, clause, counter)
        counter = counter + 1
        prepare_clause(circuit,quantum_reg, clause)
        circuit.barrier()
    #circuit.mcx(quantum_reg[n+1:counter], quantum_reg[0])
    
    #for qubit in range(n+1, counter):
        #circuit.reset(quantum_reg[qubit])
    return circuit

In [8]:
def inversion(circuit,quantum_reg, n):
    for i in range(1,n+1):
        circuit.h(quantum_reg[i])

    for i in range(1,n+1):
        circuit.x(quantum_reg[i])
    
    circuit.mcx(quantum_reg[1:n+1], quantum_reg[0])

    for i in range(1,n+1):
        circuit.x(quantum_reg[i])

    for i in range(1,n+1):
        circuit.h(quantum_reg[i])

    circuit.x(quantum_reg[0])

In [52]:
n = 3
k = 7
qubit_count = n+1
maxN = qubit_count

new_line = [[-1,-2,-3],[-1,-2,3],[-1,2,-3],[-1,2,3],[1,-2,-3],[1,-2,3],[1,2,-3]]

In [53]:
for sol in pycosat.itersolve(new_line):
    print(sol)

[-1, -2, -3]


In [61]:
qreg = QuantumRegister(n + k + 1) 
creg = ClassicalRegister(n + k + 1)
mycircuit = QuantumCircuit(qreg, creg)

for i in range(1, n+1):
    mycircuit.h(qreg[i])
    
iterations = 1
#mycircuit.x(qreg[0])
#mycircuit.h(qreg[0])

for i in range(iterations):
    #query
    oracle(mycircuit,n, k,qreg, new_line)
    mycircuit.mcx(qreg[n+1:11], qreg[0])
    mycircuit.z(qreg[0])
    mycircuit.mcx(qreg[n+1:11], qreg[0])
    for qubit in range(n+1, 11):
        mycircuit.reset(qreg[qubit])
    mycircuit.barrier()
    #inversion
    inversion_z(mycircuit,qreg, n)
    #mycircuit.barrier()


#mycircuit.h(qreg[0])
#mycircuit.x(qreg[0]) 
mycircuit.measure(qreg[0:n+1], creg[0:n+1])
job = execute(mycircuit, Aer.get_backend('qasm_simulator'), shots=1000)
#job = execute(mycircuit, backend= simulator_backend, shots=8192)
counts = job.result().get_counts(mycircuit)

# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")

00000000000 is observed 116 times
00010000000 is observed 132 times
00100000000 is observed 104 times
01000000000 is observed 125 times
01010000000 is observed 121 times
00110000000 is observed 133 times
01100000000 is observed 145 times
01110000000 is observed 124 times




In [54]:
from qiskit.circuit.library.standard_gates import ZGate

In [55]:
CCCZ = ZGate().control(2)

In [56]:
def inversion_z(circuit, quantum_reg,n):
    for i in range(1,n+1):
        circuit.h(quantum_reg[i])

    for i in range(1,n+1):
        circuit.x(quantum_reg[i])
    
    circuit.append(CCCZ, [1,2,3])

    for i in range(1,n+1):
        circuit.x(quantum_reg[i])

    for i in range(1,n+1):
        circuit.h(quantum_reg[i])

In [62]:
mycircuit.draw()

░                           ░      »
 q552_0: ───────────────────────────────░───────────────────────────░──────»
         ┌───┐┌───┐┌───┐     ┌───┐┌───┐ ░ ┌───┐┌───┐     ┌───┐┌───┐ ░ ┌───┐»
 q552_1: ┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├»
         ├───┤├───┤├───┤  │  ├───┤├───┤ ░ ├───┤├───┤  │  ├───┤├───┤ ░ ├───┤»
 q552_2: ┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├»
         ├───┤├───┤├───┤  │  ├───┤├───┤ ░ ├───┤└───┘  │  ├───┤└───┘ ░ ├───┤»
 q552_3: ┤ H ├┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├───────■──┤ X ├──────░─┤ X ├»
         └───┘└───┘└───┘┌─┴─┐├───┤└───┘ ░ └───┘       │  └───┘      ░ └───┘»
 q552_4: ───────────────┤ X ├┤ X ├──────░─────────────┼─────────────░──────»
                        └───┘└───┘      ░           ┌─┴─┐┌───┐      ░      »
 q552_5: ───────────────────────────────░───────────┤ X ├┤ X ├──────░──────»
                                        ░           └───┘└───┘      ░      »
 q552_6: ───────────────────────────────░───────────────────────────░──────»
                                        ░                           ░      »
 q552_7: ───────────────────────────────░───────────────────────────░──────»
                                        ░                           ░      »
 q552_8: ───────────────────────────────░───────────────────────────░──────»
                                        ░                           ░      »
 q552_9: ───────────────────────────────░───────────────────────────░──────»
                                        ░                           ░      »
q552_10: ───────────────────────────────░───────────────────────────░──────»
                                        ░                           ░      »
 c18: 11/══════════════════════════════════════════════════════════════════»
                                                                           »
«                              ░                           ░                »
« q552_0: ─────────────────────░───────────────────────────░────────────────»
«         ┌───┐     ┌───┐┌───┐ ░ ┌───┐┌───┐     ┌───┐┌───┐ ░ ┌───┐          »
« q552_1: ┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├───────■──»
«         └───┘  │  ├───┤└───┘ ░ ├───┤└───┘  │  ├───┤└───┘ ░ ├───┤┌───┐  │  »
« q552_2: ───────■──┤ X ├──────░─┤ X ├───────■──┤ X ├──────░─┤ X ├┤ X ├──■──»
«         ┌───┐  │  ├───┤┌───┐ ░ ├───┤       │  ├───┤      ░ ├───┤├───┤  │  »
« q552_3: ┤ X ├──■──┤ X ├┤ X ├─░─┤ X ├───────■──┤ X ├──────░─┤ X ├┤ X ├──■──»
«         └───┘  │  └───┘└───┘ ░ └───┘       │  └───┘      ░ └───┘└───┘  │  »
« q552_4: ───────┼─────────────░─────────────┼─────────────░─────────────┼──»
«                │             ░             │             ░             │  »
« q552_5: ───────┼─────────────░─────────────┼─────────────░─────────────┼──»
«              ┌─┴─┐┌───┐      ░             │             ░             │  »
« q552_6: ─────┤ X ├┤ X ├──────░─────────────┼─────────────░─────────────┼──»
«              └───┘└───┘      ░           ┌─┴─┐┌───┐      ░             │  »
« q552_7: ─────────────────────░───────────┤ X ├┤ X ├──────░─────────────┼──»
«                              ░           └───┘└───┘      ░           ┌─┴─┐»
« q552_8: ─────────────────────░───────────────────────────░───────────┤ X ├»
«                              ░                           ░           └───┘»
« q552_9: ─────────────────────░───────────────────────────░────────────────»
«                              ░                           ░                »
«q552_10: ─────────────────────░───────────────────────────░────────────────»
«                              ░                           ░                »
« c18: 11/══════════════════════════════════════════════════════════════════»
«                                                                           »
«                    ░                           ░                           ░ »
« q552_0: ───────────░───────────────────────────░───────────────────────────░─»
«  

In [12]:
"""qreg = QuantumRegister(n + k + 1) 
creg = ClassicalRegister(n + k + 1)
mycircuit = QuantumCircuit(qreg, creg)

for i in range(1, n+1):
    mycircuit.h(qreg[i])
    
iterations = 1
mycircuit.x(qreg[0])
mycircuit.h(qreg[0])

#for i in range(iterations):
    #query
oracle(mycircuit,n, k,qreg, new_line)
    #mycircuit.barrier()
    #inversion
    #inversion(mycircuit,qreg, n)
    #mycircuit.barrier()


mycircuit.h(qreg[0])
mycircuit.x(qreg[0])

job = execute(mycircuit, Aer.get_backend('statevector_simulator'), shots=1)
counts = job.result().get_statevector(mycircuit)
mycircuit.draw()
"""

"qreg = QuantumRegister(n + k + 1) \ncreg = ClassicalRegister(n + k + 1)\nmycircuit = QuantumCircuit(qreg, creg)\n\nfor i in range(1, n+1):\n    mycircuit.h(qreg[i])\n    \niterations = 1\nmycircuit.x(qreg[0])\nmycircuit.h(qreg[0])\n\n#for i in range(iterations):\n    #query\noracle(mycircuit,n, k,qreg, new_line)\n    #mycircuit.barrier()\n    #inversion\n    #inversion(mycircuit,qreg, n)\n    #mycircuit.barrier()\n\n\nmycircuit.h(qreg[0])\nmycircuit.x(qreg[0])\n\njob = execute(mycircuit, Aer.get_backend('statevector_simulator'), shots=1)\ncounts = job.result().get_statevector(mycircuit)\nmycircuit.draw()\n"

In [13]:
"""mycircuit.measure(qreg, creg)
job_2 = execute(mycircuit,  Aer.get_backend('qasm_simulator'), shots=8192)

for i,x in enumerate(counts):
    if x != 0:
        print(i, x)
    

# print the reverse of the outcome
counts = job_2.result().get_counts(mycircuit)

# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")"""

'mycircuit.measure(qreg, creg)\njob_2 = execute(mycircuit,  Aer.get_backend(\'qasm_simulator\'), shots=8192)\n\nfor i,x in enumerate(counts):\n    if x != 0:\n        print(i, x)\n    \n\n# print the reverse of the outcome\ncounts = job_2.result().get_counts(mycircuit)\n\n# print the reverse of the outcome\nfor outcome in counts:\n    reverse_outcome = \'\'\n    for i in outcome:\n        reverse_outcome = i + reverse_outcome\n    print(reverse_outcome, "is observed", counts[outcome], "times")\nprint("\n")'